# Large movie review dataset

In [3]:
import numpy as np
import tensorflow as tf
import pandas as pd
import tensorflow_hub as hub
import tensorflow_datasets as tfds

from pathlib import Path


In [4]:
root = "https://ai.stanford.edu/~amaas/data/sentiment/"
filename = "aclImdb_v1.tar.gz"

# Get and extract the tar.gz file
filepath = tf.keras.utils.get_file(filename, root + filename, extract=True, cache_dir=".")
path = Path(filepath).with_name("aclImdb")

84125825/84125825 [==============================] - 13s 0us/step


In [7]:
def review_paths(dirpath):
    return [str(path) for path in dirpath.glob("*.txt")]

train_pos = review_paths(path / "train" / "pos")
train_neg = review_paths(path / "train" / "neg")
test_valid_pos = review_paths(path / "test" / "pos")
test_valid_neg = review_paths(path / "test" / "neg")

len(train_pos), len(train_neg), len(test_valid_pos), len(test_valid_neg)


(12500, 12500, 12500, 12500)

In [9]:
# Splitting test set into test and valid set

np.random.shuffle(test_valid_pos)
np.random.shuffle(test_valid_neg)

test_pos = test_valid_pos[:5000]
test_neg = test_valid_neg[:5000]

valid_pos = test_valid_pos[5000:]
valid_neg = test_valid_neg[5000:]


In [11]:
# Load the datasets into a tf.Dataset

def imdb_dataset(filepaths_positive, filepaths_negative):
    reviews = []
    labels = []
    for filepaths, label in ((filepaths_negative, 0), (filepaths_positive, 1)):
        for filepath in filepaths:
            with open(filepath) as review_file:
                reviews.append(review_file.read())
            labels.append(label)

    return tf.data.Dataset.from_tensor_slices(
        (tf.constant(reviews), tf.constant(labels))
      )


batch_size = 32
train_set = imdb_dataset(train_pos, train_neg).shuffle(25000, seed=42)
train_set = train_set.batch(batch_size).prefetch(1)

valid_set = imdb_dataset(valid_pos, valid_neg).batch(batch_size).prefetch(1)
test_set = imdb_dataset(test_pos, test_neg).batch(batch_size).prefetch(1)

In [16]:
max_tokens = 1000

# Discard the labels for the text vectorization
sample_reviews = train_set.map(lambda review, label: review)
text_vectorization = tf.keras.layers.TextVectorization(max_tokens=max_tokens, output_mode="tf_idf")
text_vectorization.adapt(sample_reviews)

In [14]:
text_vectorization.get_vocabulary()[:10]

['[UNK]', 'the', 'and', 'a', 'of', 'to', 'is', 'in', 'it', 'i']

In [18]:
# Creating a model 

tf.random.set_seed(42)
model = tf.keras.Sequential([
    text_vectorization,
    tf.keras.layers.Dense(100, activation="relu"),
    tf.keras.layers.Dense(1, activation="sigmoid"),
])
model.compile(loss="binary_crossentropy", optimizer="nadam", metrics=["accuracy"])
model.fit(train_set, epochs=5, validation_data=valid_set)

Epoch 1/5
782/782 [==============================] - 4s 3ms/step - loss: 0.4360 - accuracy: 0.8204 - val_loss: 0.5932 - val_accuracy: 0.7832
Epoch 2/5
782/782 [==============================] - 3s 3ms/step - loss: 0.3642 - accuracy: 0.8540 - val_loss: 0.3614 - val_accuracy: 0.8541
Epoch 3/5
782/782 [==============================] - 3s 3ms/step - loss: 0.3114 - accuracy: 0.8711 - val_loss: 0.5593 - val_accuracy: 0.7909
Epoch 4/5
782/782 [==============================] - 2s 3ms/step - loss: 0.2533 - accuracy: 0.8967 - val_loss: 0.4186 - val_accuracy: 0.8358
Epoch 5/5
782/782 [==============================] - 3s 3ms/step - loss: 0.1894 - accuracy: 0.9260 - val_loss: 0.4360 - val_accuracy: 0.8347
